# RAG with Llama Stack Primitives

This notebook demonstrates building a **Retrieval-Augmented Generation (RAG)** system using **pure Llama Stack primitives** - no frameworks like LangChain or CrewAI.

## Agentic RAG Approach

This notebook shows the **Agentic RAG loop**:

1. **Query** → User asks a question
2. **Search** → Semantic search in vector store
3. **Extract** → Get relevant document chunks
4. **Generate** → Pass context + query to LLM
5. **Answer** → Return generated response

## Architecture

```
┌─────────────────────────────────────────────┐
│         User Query                          │
└────────┬────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────┐
│  Vector Search                              │
│  (client.vector_stores.search)              │
└────────┬────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────┐
│  Context Extraction                         │
│  (Extract relevant chunks)                  │
└────────┬────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────┐
│  Llama Stack Responses API                  │
│  (client.responses.create)                  │
└────────┬────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────┐
│         Final Answer                        │
└─────────────────────────────────────────────┘
```

## Configuration

This notebook uses environment variables from `.env` file in the project root.
Create your own `.env` file based on `.env.example`.

## Install Required Packages

Install Llama Stack client (version must match server version):

In [ ]:
!pip3 install -q "llama-stack-client==0.2.22" "python-dotenv"

## Import Dependencies

In [ ]:
# Core imports
import os
import sys
from pathlib import Path
from io import BytesIO
from dotenv import load_dotenv

# Llama Stack client
from llama_stack_client import LlamaStackClient

# Load environment variables from project root
project_root = Path.cwd().parent.parent if 'assets/notebooks' in str(Path.cwd()) else Path.cwd()
env_path = project_root / '.env'
load_dotenv(env_path)

print(f"📁 Loading environment from: {env_path}")
print(f"✅ .env file {'found' if env_path.exists() else 'NOT FOUND'}")

# Verify kernel
print(f"\n🐍 Python: {sys.executable}")
if ".venv" in sys.executable:
    print("✅ Using project venv - CORRECT!")
else:
    print("⚠️  Using global Python - Consider switching kernel!")
    print("   Click 'Select Kernel' → Choose 'Python (byo-agentic-framework)')")

## Configure Llama Stack Connection

Create client using Llama Stack's base URL:

In [ ]:
# Get configuration from environment variables
LLAMA_STACK_BASE_URL = os.getenv("LLAMA_STACK_BASE_URL")
INFERENCE_MODEL = os.getenv("INFERENCE_MODEL")
LLAMA_STACK_OPENAI_ENDPOINT = os.getenv("LLAMA_STACK_OPENAI_ENDPOINT")

print("🌐 Llama Stack Configuration:")
print(f"   Base URL: {LLAMA_STACK_BASE_URL}")
print(f"   Model: {INFERENCE_MODEL}")
print(f"   OpenAI Endpoint: {LLAMA_STACK_OPENAI_ENDPOINT}")

# Create client using the base URL
client = LlamaStackClient(base_url=LLAMA_STACK_BASE_URL)
print("\n✅ Llama Stack client created")

## Vector Store Setup

Create a vector store with sample Acme company documentation.

### Step 1: Prepare Documents

We'll use three simple documents about company policies:

In [ ]:
print("📚 Preparing Acme documentation...\n")

# Sample documents for RAG
docs = [
    ("Acme ships globally in 3-5 business days.", {"title": "Shipping Policy"}),
    ("Returns are accepted within 30 days of purchase.", {"title": "Returns Policy"}),
    ("Support is available 24/7 via chat and email.", {"title": "Support"}),
]

print("📄 Documents to upload:")
for content, metadata in docs:
    print(f"   • {metadata['title']}: {content}")

print(f"\n✅ {len(docs)} documents prepared")

### Step 2: Upload Files to Llama Stack

Upload documents using the Files API:

In [ ]:
file_ids = []
print("📤 Uploading documents to Llama Stack...\n")

for content, metadata in docs:
    # Create in-memory file from document text
    with BytesIO(content.encode()) as file_buffer:
        # Set filename for the buffer
        file_buffer.name = f"{metadata['title'].replace(' ', '_').lower()}.txt"
        
        # Upload to Llama Stack
        file_response = client.files.create(
            file=file_buffer,
            purpose="assistants"
        )
        
        print(f"   ✅ Uploaded: {metadata['title']}")
        print(f"      File ID: {file_response.id}")
        
        file_ids.append(file_response.id)

print(f"\n✅ All {len(file_ids)} files uploaded successfully")

### Step 3: Create Vector Store

Create a vector store with the uploaded files:

In [ ]:
print("🔧 Creating vector store...\n")

vector_store = client.vector_stores.create(
    name="acme_docs",
    file_ids=file_ids,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    embedding_dimension=384,
    provider_id="milvus"
)

print("✅ Vector store created successfully!")
print(f"   ID: {vector_store.id}")
print(f"   Name: {vector_store.name}")
print(f"   Embedding Model: sentence-transformers/all-MiniLM-L6-v2")
print(f"   Dimensions: 384")
print(f"   Provider: Milvus")

## Test Direct Vector Search

Test semantic search by querying the vector store directly:

In [ ]:
print("🔍 Testing vector search...\n")

test_query = "How long does shipping take?"
print(f"Query: {test_query}\n")

# Perform semantic search
search_response = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query=test_query,
    max_num_results=2
)

# Display results
print("📊 Search Results:\n")
for i, result in enumerate(search_response.data, 1):
    content = result.content[0].text
    filename = result.filename if hasattr(result, 'filename') else 'Unknown'
    print(f"   [{i}] {filename}")
    print(f"       Content: {content}\n")

print("✅ Vector search working correctly!")

## Agentic RAG Function

Define a function that implements the Agentic RAG loop:

1. Search the vector store for relevant chunks
2. Extract context from search results
3. Build a prompt with context + query
4. Call Responses API to generate answer

In [ ]:
def agentic_rag(client, vector_store_id, query, model, max_results=3):
    """
    Perform Agentic RAG: search → extract context → generate answer
    
    Args:
        client: LlamaStackClient instance
        vector_store_id: ID of the vector store to search
        query: User's question
        model: Model ID for inference
        max_results: Number of chunks to retrieve
    
    Returns:
        dict with query, context, answer, and metadata
    """
    # 1. Search vector store for relevant chunks
    search_results = client.vector_stores.search(
        vector_store_id=vector_store_id,
        query=query,
        max_num_results=max_results
    )
    
    # 2. Extract context from search results using traditional loop
    context_chunks = []
    for result in search_results.data:
        text = result.content[0].text
        context_chunks.append(text)
    
    context = "\n\n".join(context_chunks)
    
    # 3. Build prompt with context
    prompt = f"""Use the following context to answer the question.

Context:
{context}

Question: {query}

Answer based only on the context provided above."""
    
    # 4. Call Responses API to generate answer
    response = client.responses.create(
        model=model,
        input=prompt
    )
    
    return {
        "query": query,
        "context": context,
        "answer": response.output_text,
        "num_chunks": len(context_chunks)
    }

print("✅ Agentic RAG function defined")

## Example 1: Shipping Query

Test the RAG system with a shipping-related question:

In [ ]:
print("\n" + "="*60)
print("🚀 Example 1: Shipping Query")
print("="*60 + "\n")

query = "How long does shipping take?"
print(f"❓ Query: {query}\n")

# Perform agentic RAG
result = agentic_rag(
    client=client,
    vector_store_id=vector_store.id,
    query=query,
    model=INFERENCE_MODEL,
    max_results=3
)

# Display results
print(f"🔍 Retrieved {result['num_chunks']} relevant chunks\n")
print("📄 Context:")
print("-" * 60)
print(result['context'])
print("-" * 60)

print("\n💡 Answer:")
print("=" * 60)
print(result['answer'])
print("=" * 60)

## Example 2: Returns Policy Query

Test with a question about return policies:

In [ ]:
print("\n" + "="*60)
print("🚀 Example 2: Returns Policy Query")
print("="*60 + "\n")

query = "Can I return a product after 40 days?"
print(f"❓ Query: {query}\n")

# Perform agentic RAG
result = agentic_rag(
    client=client,
    vector_store_id=vector_store.id,
    query=query,
    model=INFERENCE_MODEL,
    max_results=3
)

# Display results
print(f"🔍 Retrieved {result['num_chunks']} relevant chunks\n")
print("📄 Context:")
print("-" * 60)
print(result['context'])
print("-" * 60)

print("\n💡 Answer:")
print("=" * 60)
print(result['answer'])
print("=" * 60)

## Example 3: Support Query

Test with a question about customer support:

In [ ]:
print("\n" + "="*60)
print("🚀 Example 3: Support Query")
print("="*60 + "\n")

query = "How can I contact support?"
print(f"❓ Query: {query}\n")

# Perform agentic RAG
result = agentic_rag(
    client=client,
    vector_store_id=vector_store.id,
    query=query,
    model=INFERENCE_MODEL,
    max_results=3
)

# Display results
print(f"🔍 Retrieved {result['num_chunks']} relevant chunks\n")
print("📄 Context:")
print("-" * 60)
print(result['context'])
print("-" * 60)

print("\n💡 Answer:")
print("=" * 60)
print(result['answer'])
print("=" * 60)

---

## Summary

We have successfully built an Agentic RAG system using **pure Llama Stack primitives and Responses API** - no frameworks needed!

### What We Demonstrated:

✅ **Vector Store Creation** - Created a vector database with embedded documents  
✅ **Document Upload** - Uploaded files using the Files API  
✅ **Semantic Search** - Performed vector similarity search  
✅ **Context Extraction** - Retrieved relevant document chunks  
✅ **Response Generation** - Used Responses API for final answer  

### Key Llama Stack Primitives Used:

| API | Purpose | Method |
|-----|---------|--------|
| **Files API** | Upload documents | `client.files.create()` |
| **Vector Stores API** | Create vector database | `client.vector_stores.create()` |
| **Search API** | Semantic search | `client.vector_stores.search()` |
| **Responses API** | Generate answers | `client.responses.create()` |

### Agentic RAG Workflow:

```
1. User Query
   ↓
2. vector_stores.search() → Find relevant chunks
   ↓
3. Extract context from search results
   ↓
4. Build prompt with context + query
   ↓
5. responses.create() → Generate answer
   ↓
6. Return final answer
```